<a href="https://colab.research.google.com/github/Imesha599/Automated-handwritten-Prescription-Medication-Suggestion/blob/main/Medisug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install tabulate
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.0 MB/s eta 0:00:00


In [3]:
import requests
import openai
from tabulate import tabulate
import textwrap
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import json

# Set OpenAI API key
openai.api_key = ""

# OpenFDA API key
openfda_api_key = ""

# Fetch medication names from OpenFDA
def fetch_medication_names():
    url = f"https://api.fda.gov/drug/label.json?search=openfda.generic_name:*+openfda.brand_name:*&limit=1000&api_key={openfda_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        medication_names = set()
        for result in data.get('results', []):
            if 'openfda' in result:
                # Add generic names
                if 'generic_name' in result['openfda']:
                    medication_names.update(result['openfda']['generic_name'])
                # Add brand names
                if 'brand_name' in result['openfda']:
                    medication_names.update(result['openfda']['brand_name'])
        return list(medication_names)
    else:
        print(f"API error: {response.status_code}")
        return []

# Fetch medication information from OpenFDA
def get_medication_info(medication_name):
    url = f"https://api.fda.gov/drug/label.json?search=openfda.generic_name:{medication_name}+openfda.brand_name:{medication_name}&limit=1&api_key={openfda_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            return data['results'][0]
        else:
            print("Medication information not found.")
            return None
    else:
        print(f"API error: {response.status_code}")
        return None

# Function to fetch medication side effects from OpenFDA
def get_medication_side_effects(medication_name):
    url = f"https://api.fda.gov/drug/event.json?search=patient.drug.openfda.generic_name:{medication_name}&limit=1&api_key={openfda_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            side_effects = data['results'][0].get('patient', {}).get('reaction', [])
            side_effect_descriptions = [reaction.get('reactionmeddrapt', 'Unknown') for reaction in side_effects]
            return ', '.join(side_effect_descriptions) if side_effect_descriptions else 'No side effects found.'
        else:
            return "No side effects information found."
    else:
        print(f"API error: {response.status_code}")
        return "Error retrieving side effects information."

# Function to generate text using GPT-3.5 Turbo
def generate_text(prompt, role="system"):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": role, "content": prompt}],
            max_tokens=150
        )
        return response['choices'][0]['message']['content'].strip()
    except openai.error.OpenAIError as e:
        return f"An error occurred: {str(e)}"

# Function to detect the type of entered Prescription Name
def detect_prescription_name_type(text):
    if text.isupper():
        return "Abbreviation"
    elif len(text) <= 5:
        return "Partial Name"
    else:
        return "Misspelling"

# Function to map abbreviations to full names dynamically
def map_abbreviations(medication_name, medication_names):
    matched = process.extractOne(medication_name, medication_names, scorer=fuzz.ratio)
    if matched and matched[1] > 80:
        return matched[0]
    else:
        return medication_name

# Function to handle misspellings dynamically
def handle_misspellings(medication_name, medication_names):
    matched = process.extractOne(medication_name, medication_names, scorer=fuzz.ratio)
    if matched and matched[1] > 80:
        return matched[0]
    else:
        return generate_text(f"Correct the medication name '{medication_name}'.")

# Function to predict full medication name based on partial name
def predict_full_name(partial_name, medication_names):
    matched = process.extractOne(partial_name, medication_names, scorer=fuzz.ratio)
    if matched and matched[1] > 80:
        return matched[0]
    return partial_name

# Function to process prescription text and handle various issues
def process_prescription_text(prescription_text, medication_names):
    # Split the input into prescription name and usage instruction
    try:
        prescription_name, usage_instruction = prescription_text.split(',', 1)
        prescription_name = prescription_name.strip()
        usage_instruction = usage_instruction.strip()
    except ValueError:
        print("Error: Please enter the prescription name and usage instruction separated by a comma.")
        return None, None, None, None, None, None

    text_type = detect_prescription_name_type(prescription_name)

    # Correct abbreviations
    corrected_medication = map_abbreviations(prescription_name, medication_names)

    # Handle misspellings dynamically
    corrected_medication = handle_misspellings(corrected_medication, medication_names)

    # Predict full name from partial name
    corrected_medication = predict_full_name(corrected_medication, medication_names)

    # Fetch detailed information from OpenFDA
    medication_info = get_medication_info(corrected_medication)

    # Generate brief text about the medication using ChatGPT
    description_prompt = (
        f"Provide description for the medication '{corrected_medication}' "
        "including its common uses and how it works. and focus only on key points."
    )
    detailed_description = generate_text(description_prompt, role="user")

    # Generate concise usage instructions using ChatGPT
    usage_generation_prompt = (
        f"Expand the usage instruction '{usage_instruction}' for the medication '{corrected_medication}' "
        "in one line, ensuring to use 'tablet' instead of 'tab' and omit phrases like 'Take one'."
    )
    expanded_usage_instruction = generate_text(usage_generation_prompt, role="user")

    # Generate side effects information
    side_effects = get_medication_side_effects(corrected_medication)

    # If side effects are not found, use GPT-3.5 Turbo to generate them
    if "No side effects information found." in side_effects or "Error retrieving side effects information." in side_effects:
        side_effects_prompt = f"List possible side effects for the medication '{corrected_medication}'."
        side_effects = generate_text(side_effects_prompt, role="user")

    # Suggested output combining corrected medication name and expanded usage instruction
    suggested_output = f"{corrected_medication} {expanded_usage_instruction}"

    return text_type, corrected_medication, detailed_description, expanded_usage_instruction, side_effects, suggested_output

# Function to collect feedback from the user
def collect_feedback(suggested_output):
    feedback_prompt = textwrap.shorten(f"Do you agree with the suggested output? (yes/no): ", width=100, placeholder="...")
    feedback = input(feedback_prompt).strip().lower()
    if feedback == 'no':
        reason = input("Please explain why you disagree with the suggested output: ").strip()
        return reason
    return None

# Function to store feedback in a file
def store_feedback(prescription_text, usage_instruction, reason):
    feedback_data = {
        "prescription_text": prescription_text,
        "usage_instruction": usage_instruction,
        "reason": reason
    }
    try:
        with open("feedback.json", "a") as f:
            json.dump(feedback_data, f)
            f.write("\n")
    except IOError as e:
        print(f"Error storing feedback: {e}")

def main():
    # Fetch medication names from OpenFDA
    medication_names = fetch_medication_names()

    # Welcome message
    print("Welcome to the Automated Prescription Detection System!")
    print("Our System helps you identify and correct medication names, as well as expand usage instructions.")
    print()

    # Prompt the user for prescription text
    prescription_text = input("Enter the prescription Text: ")

    # Validate the input
    if not prescription_text:
        print("Error: Prescription text must be provided.")
        return

    # Process the prescription text
    text_type, corrected_medication, detailed_description, expanded_usage_instruction, side_effects, suggested_output = process_prescription_text(prescription_text, medication_names)

    # Ensure valid processing before displaying results
    if None in [text_type, corrected_medication, detailed_description, expanded_usage_instruction, side_effects, suggested_output]:
        print("Error: Unable to process the input. Please try again.")
        return

    # Prepare data for tabular display
    data = [
        ["Entered Prescription Name Type", text_type],
        ["Corrected Medication Name", "\n".join(textwrap.wrap(corrected_medication, width=100))],
        ["Description", "\n".join(textwrap.wrap(detailed_description, width=100))],
        ["Expanded Usage Instruction", "\n".join(textwrap.wrap(expanded_usage_instruction, width=100))],
        ["Side Effects", "\n".join(textwrap.wrap(side_effects, width=100))],
        ["Suggested Output", "\n".join(textwrap.wrap(suggested_output, width=100))]
    ]

    # Print the results in a table
    print("\n--- Results ---")
    print(tabulate(data, headers=["Field", "Value"], tablefmt="grid"))

    # Collect feedback from the user
    reason = collect_feedback(suggested_output)
    if reason:
        prescription_name, usage_instruction = prescription_text.split(',', 1)
        store_feedback(prescription_name.strip(), usage_instruction.strip(), reason)
        print("\nThank you for your feedback. Your input is valuable in helping us enhance the system.")

    # Provide additional information
    print("\nThank you for using the Automated Prescription Detection System!")
    print("If you encounter any issues or have feedback, please contact support.")

# Run the main function
if __name__ == "__main__":
    main()


Welcome to the Automated Prescription Detection System!
Our System helps you identify and correct medication names, as well as expand usage instructions.

Enter the prescription Text: APAP,100mg tab pc
API error: 500
API error: 500

--- Results ---
+--------------------------------+------------------------------------------------------------------------------------------------------+
| Field                          | Value                                                                                                |
+================================+======================================================================================================+
| Entered Prescription Name Type | Abbreviation                                                                                         |
+--------------------------------+------------------------------------------------------------------------------------------------------+
| Corrected Medication Name      | Corrected: acetaminophen  